In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Load the movie metadata file
movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# View first few rows
movies[['title', 'overview']].head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [3]:
# Replace NaN with empty string
movies['overview'] = movies['overview'].fillna('')

In [4]:
# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

# Check shape
print(tfidf_matrix.shape)  # Example: (45466, 75827)

(45466, 75827)


In [5]:
# Calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
# Reset index to access title directly
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [7]:
def recommend(title, cosine_sim=cosine_sim):
    if title not in indices:
        return "Movie not found in dataset."
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [8]:
import gradio as gr

# Define Gradio input/output function
def gradio_recommend(title):
    recommendations = recommend(title)
    return "\n".join(recommendations)

# Create Gradio interface
demo = gr.Interface(
    fn=gradio_recommend,
    inputs=gr.Textbox(label="Enter Movie Title"),
    outputs=gr.Textbox(label="Recommended Movies"),
    title="🎬 Movie Recommendation System",
    description="Enter the name of a movie and get 10 similar movie recommendations!",
)

# Launch locally
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
